import di tutte le libreri utilizzate e inizializzazione delle costanti

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, Dropdown, IntText, Checkbox
import csv
import os
import random
import sys
import heapq

# Inizializza un contatore
file_counter = 0

In [4]:
# Creare un grafo casuale (nel seguente esempio, un grafo di Erdos-Renyi)
num_nodes = 10  # Numero di nodi nel grafo
probability = 0.3  # Probabilità di connessione tra nodi

G = nx.erdos_renyi_graph(num_nodes, probability, directed=True)

# Assicurarsi che ogni nodo abbia almeno un collegamento
for node in G.nodes():
    if G.out_degree(node) == 0:
        # Trova un nodo diverso da collegare
        target_node = node
        while target_node == node or G.has_edge(node, target_node):
            target_node = random.choice(list(G.nodes()))

        G.add_edge(node, target_node)

# Incrementare il contatore ad ogni esecuzione
file_counter += 1

# Creare una cartella per i file se non esiste già
output_folder_graphs = 'graphs'
#output_folder_nodes = 'nodes'

if not os.path.exists(output_folder_graphs):
    os.makedirs(output_folder_graphs)
    
#f not os.path.exists(output_folder_nodes):
#    os.makedirs(output_folder_nodes)

# Creare nomi di file basati sul contatore
#nodes_filename = os.path.join(output_folder_nodes, f'grafo_nodi_{file_counter}.csv')
edges_filename = os.path.join(output_folder_graphs, f'grafo_archi_{file_counter}.csv')

# Aggiungo un attributo "capacità" casuale a ciascun nodo
for node in G.nodes():
    G.nodes[node]['capacità'] = random.randint(1, 10)  # Imposta una capacità casuale da 1 a 10 (puoi personalizzare l'intervallo)

# Creare un file CSV e scrivere gli archi diretti e le capacità
with open(edges_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Nodo1', 'Nodo2', 'Capacità'])
    for edge in G.edges():
        writer.writerow([edge[0], edge[1], G.nodes[edge[0]]['capacità']])


Visualizzare i grafi

In [5]:
file = os.listdir("./graphs/")
file_list = ["./graphs/" + f for f in file]
_ = file_list.pop(0)  # Elimina il primo file che trova (file autogenerato che non fa parte dei grafi)

def fromPathToGraph(path: list):
    G = nx.DiGraph()  # Utilizza DiGraph per grafo orientato
    for index in range(0, len(path) - 1):
        G.add_edge(path[index], path[index + 1])
    return G

@interact(file=Dropdown(options=file_list, description='Graph:'),
          show=Checkbox(description="Plot the graph"))
def selectGraph(file, show): 
    df = pd.read_csv(file, names=["Node1", "Node2", "Capacità"])
    df = df.iloc[1:]  # Elimina la prima riga
    G = nx.DiGraph()  # Utilizza DiGraph per grafo orientato
    Graphtype = nx.DiGraph()  # Utilizza DiGraph per grafo orientato
    G = nx.from_pandas_edgelist(df, source="Node1", target="Node2", edge_attr='Capacità', create_using=Graphtype)
    if show:
        pos = nx.spring_layout(G)  # Calcola la posizione dei nodi
        edge_labels = {(u, v): d["Capacità"] for u, v, d in G.edges(data=True)}  # Crea un dizionario di etichette degli archi
        
        nx.draw(G, pos, with_labels=True, node_size=700, node_color="skyblue", font_size=10)
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)  # Aggiungi etichette degli archi


interactive(children=(Dropdown(description='Graph:', options=('./graphs/grafo_archi_6.csv', './graphs/grafo_ar…

Cycle cancelling

In [9]:
# Chiedi all'utente il nome del file CSV contenente il grafo
file_name = input("Inserisci il nome del file CSV contenente il grafo: ")

output_folder_graphs = 'graphs'
graph_file_name = os.path.join(output_folder_graphs, f'grafo_archi_{file_name}.csv')

# Carica il grafo da un file CSV
G = nx.DiGraph()  # Grafo diretto
with open(graph_file_name, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        source = int(row['Nodo1'])
        target = int(row['Nodo2'])
        capacity = int(row['Capacità'])
        G.add_edge(source, target, capacity=capacity)

# Calcola il flusso minimo utilizzando l'algoritmo del ciclo di cancellazione
flow_value, flow_dict = nx.maximum_flow(G, 0, 9)  # Inserisci i nodi sorgente e destinazione desiderati

# Restituisci il risultato del flusso minimo
print(f'Valore del flusso minimo: {flow_value}')
print('Flusso minimo:')
for source_node, target_node in G.edges():
    print(f'Nodo {source_node} -> Nodo {target_node}: {flow_dict[source_node][target_node]}')


Valore del flusso minimo: 13
Flusso minimo:
Nodo 0 -> Nodo 1: 2
Nodo 0 -> Nodo 2: 3
Nodo 0 -> Nodo 8: 8
Nodo 1 -> Nodo 5: 0
Nodo 1 -> Nodo 6: 2
Nodo 1 -> Nodo 8: 0
Nodo 1 -> Nodo 9: 2
Nodo 2 -> Nodo 1: 0
Nodo 2 -> Nodo 3: 0
Nodo 2 -> Nodo 5: 0
Nodo 2 -> Nodo 8: 0
Nodo 2 -> Nodo 9: 3
Nodo 8 -> Nodo 1: 2
Nodo 8 -> Nodo 4: 2
Nodo 8 -> Nodo 6: 2
Nodo 8 -> Nodo 7: 2
Nodo 5 -> Nodo 8: 0
Nodo 6 -> Nodo 2: 0
Nodo 6 -> Nodo 3: 0
Nodo 6 -> Nodo 7: 4
Nodo 9 -> Nodo 3: 0
Nodo 9 -> Nodo 6: 0
Nodo 3 -> Nodo 2: 0
Nodo 3 -> Nodo 8: 0
Nodo 4 -> Nodo 2: 0
Nodo 4 -> Nodo 5: 0
Nodo 4 -> Nodo 6: 0
Nodo 4 -> Nodo 9: 2
Nodo 7 -> Nodo 9: 6


Questa implementazione mostra un'idea di base di come utilizzare nx.maximum_flow e il cycle cancelling per calcolare il flusso minimo in un grafo.

In [ ]:
def cycle_canceling(graph, source, sink):
    # Inizializza il flusso a zero
    flow_value, flow_dict = nx.maximum_flow(graph, source, sink)
    
    # Calcola il costo residuo (differenza tra costi degli archi e flusso)
    residual_graph = graph.copy()
    for u, v, attr in residual_graph.edges(data=True):
        attr['capacity'] -= flow_dict[u][v]
        attr['cost'] = -attr['cost']
    
    # Ripeti fino a quando ci sono archi con capacità residua positiva e costo negativo
    while True:
        # Trova un arco con capacità residua positiva e costo negativo
        edge_to_update = None
        for u, v, attr in residual_graph.edges(data=True):
            if attr['capacity'] > 0 and attr['cost'] < 0:
                edge_to_update = (u, v)
                break
        
        # Se non ci sono più archi da aggiornare, esci dal ciclo
        if edge_to_update is None:
            break
        
        # Trova un cammino dal nodo sorgente al nodo destinazione nel grafo residuo
        try:
            path = nx.shortest_path(residual_graph, source, sink, weight='cost')
        except nx.NetworkXNoPath:
            break
        
        # Calcola la capacità residua minima lungo il cammino
        min_capacity = min(residual_graph[u][v]['capacity'] for u, v in zip(path, path[1:]))
        
        # Aggiorna il flusso lungo il cammino
        for u, v in zip(path, path[1:]):
            residual_graph[u][v]['capacity'] -= min_capacity
            residual_graph[v][u]['capacity'] += min_capacity
        
        # Calcola il costo residuo
        for u, v, attr in residual_graph.edges(data=True):
            attr['cost'] = -attr['cost']
        
        # Aggiorna il flusso massimo
        flow_value += min_capacity
    
    return flow_value, flow_dict

# Esempio di utilizzo
graph = nx.DiGraph()

# Aggiungi nodi e archi con capacità e costi
# ...

source = 0
sink = 9

flow_value, flow_dict = cycle_canceling(graph, source, sink)
print(f'Valore del flusso minimo: {flow_value}')
print('Flusso minimo:')
for u, v in graph.edges():
    print(f'Nodo {u} -> Nodo {v}: {flow_dict[u][v]}')


successive Shortest Path (SSP) 

In [12]:
# Chiedi all'utente il nome del file CSV contenente il grafo
file_name = input("Inserisci il nome del file CSV contenente il grafo: ")

output_folder_graphs = 'graphs'
graph_file_name = os.path.join(output_folder_graphs, f'grafo_archi_{file_name}.csv')

# Carica il grafo da un file CSV
G = nx.DiGraph()  # Grafo diretto
with open(graph_file_name, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        source = int(row['Nodo1'])
        target = int(row['Nodo2'])
        capacity = int(row['Capacità'])
        G.add_edge(source, target, capacity=capacity)

# Adatta il grafo in modo che tutti i costi siano positivi
for u, v, attr in G.edges(data=True):
    attr['weight'] = -attr['capacity']

# Calcola il flusso minimo utilizzando l'algoritmo di flusso massimo (Successive Shortest Path)
source = 0  # Nodo sorgente
target = 9  # Nodo destinazione

flow_dict = nx.min_cost_flow(G)

# Ottieni il valore del flusso minimo sommando i flussi uscenti dal nodo sorgente (0)
flow_value = sum(flow_dict[source][neighbor] for neighbor in G.neighbors(source))

# Restituisci il risultato del flusso minimo
print(f'Valore del flusso minimo: {flow_value}')
print('Flusso minimo:')
for u, v in G.edges():
    print(f'Nodo {u} -> Nodo {v}: {flow_dict[u][v]}')


Valore del flusso minimo: 0
Flusso minimo:
Nodo 0 -> Nodo 1: 0
Nodo 0 -> Nodo 2: 0
Nodo 0 -> Nodo 8: 0
Nodo 1 -> Nodo 5: 1
Nodo 1 -> Nodo 6: 2
Nodo 1 -> Nodo 8: 0
Nodo 1 -> Nodo 9: 2
Nodo 2 -> Nodo 1: 3
Nodo 2 -> Nodo 3: 3
Nodo 2 -> Nodo 5: 2
Nodo 2 -> Nodo 8: 0
Nodo 2 -> Nodo 9: 2
Nodo 8 -> Nodo 1: 2
Nodo 8 -> Nodo 4: 2
Nodo 8 -> Nodo 6: 2
Nodo 8 -> Nodo 7: 2
Nodo 5 -> Nodo 8: 3
Nodo 6 -> Nodo 2: 4
Nodo 6 -> Nodo 3: 3
Nodo 6 -> Nodo 7: 4
Nodo 9 -> Nodo 3: 5
Nodo 9 -> Nodo 6: 5
Nodo 3 -> Nodo 2: 6
Nodo 3 -> Nodo 8: 5
Nodo 4 -> Nodo 2: 0
Nodo 4 -> Nodo 5: 0
Nodo 4 -> Nodo 6: 2
Nodo 4 -> Nodo 9: 0
Nodo 7 -> Nodo 9: 6


implementazione ssp

In [8]:
class Edge:
    def __init__(self, start, end, capacity, cost):
        self.start = start
        self.end = end
        self.capacity = capacity
        self.cost = cost
        self.flow = 0

def add_edge(graph, start, end, capacity, cost):
    forward_edge = Edge(start, end, capacity, cost)
    backward_edge = Edge(end, start, 0, -cost)  # Backward edge with 0 capacity and negative cost
    graph[start].append(forward_edge)
    graph[end].append(backward_edge)

def successive_shortest_path(graph, source, sink):
    n = len(graph)
    INF = sys.maxsize
    dist = [INF] * n
    prev = [None] * n

    max_flow = 0
    min_cost = 0

    while True:
        dist[source] = 0
        hq = [(0, source)]  # Priority queue for Dijkstra's algorithm
        while hq:
            d, u = heapq.heappop(hq)
            if d > dist[u]:
                continue
            for edge in graph[u]:
                if edge.capacity - edge.flow > 0 and dist[edge.end] > dist[u] + edge.cost:
                    dist[edge.end] = dist[u] + edge.cost
                    prev[edge.end] = edge
                    heapq.heappush(hq, (dist[edge.end], edge.end))

        if dist[sink] == INF:
            break

        augmenting_path_flow = INF
        node = sink
        while node != source:
            edge = prev[node]
            augmenting_path_flow = min(augmenting_path_flow, edge.capacity - edge.flow)
            node = edge.start

        max_flow += augmenting_path_flow
        min_cost += augmenting_path_flow * dist[sink]

        node = sink
        while node != source:
            edge = prev[node]
            edge.flow += augmenting_path_flow
            for backward_edge in graph[edge.end]:
                if backward_edge.end == edge.start:
                    backward_edge.flow -= augmenting_path_flow
                    break
            node = edge.start

    return max_flow, min_cost

if __name__ == "__main__":
    # Esempio di utilizzo
    num_nodi = 4
    sorgente = 0
    pozzo = 3
    grafo = [[] for _ in range(num_nodi)]

    add_edge(grafo, 0, 1, 2, 1)
    add_edge(grafo, 0, 2, 1, 2)
    add_edge(grafo, 1, 2, 1, 1)
    add_edge(grafo, 1, 3, 2, 3)
    add_edge(grafo, 2, 3, 2, 1)

    flusso_massimo, costo_minimo = successive_shortest_path(grafo, sorgente, pozzo)
    print(f"Flusso Massimo: {flusso_massimo}")
    print(f"Costo Minimo: {costo_minimo}")


KeyboardInterrupt: 

possiamo notare come partendo da uno stesso grafo i due algoritmi restituiscano un flusso minimo differente

È possibile che l'algoritmo del cycle cancelling e l'algoritmo Successive Shortest Path (SSP) restituiscano valori diversi per il flusso minimo in un grafo specifico. Questo può essere dovuto a diversi fattori, tra cui la struttura del grafo, i costi associati agli archi e le capacità dei nodi.

L'algoritmo del cycle cancelling è specificamente progettato per risolvere il problema del flusso minimo in grafi a costo minimo, mentre SSP può essere utilizzato su una gamma più ampia di problemi di flusso. La differenza nei risultati potrebbe essere dovuta a come ciascun algoritmo gestisce determinate configurazioni di grafo o costi negativi.

Se il tuo obiettivo è ottenere il flusso minimo in un grafo specifico, è importante considerare quale algoritmo è più adatto per il tuo caso specifico e quali parametri e configurazioni stai utilizzando. Puoi anche esaminare la documentazione di NetworkX e le implementazioni degli algoritmi per comprendere meglio le differenze nei risultati e nelle condizioni d'uso.

Sì, è possibile ottenere due flussi minimi diversi all'interno dello stesso grafo utilizzando l'algoritmo di Cycle Canceling (CC) e l'algoritmo Successive Shortest Paths (SSP). Entrambi questi algoritmi sono utilizzati per trovare il flusso minimo all'interno di un grafo di rete, ma possono condurre a soluzioni diverse, a seconda dell'implementazione o delle scelte specifiche fatte durante l'esecuzione.

Ecco alcune ragioni per cui potresti ottenere risultati diversi utilizzando CC e SSP sullo stesso grafo:

1. Inizializzazione del flusso: L'inizializzazione del flusso iniziale può variare tra le implementazioni di CC e SSP. Un'inizializzazione diversa può portare a flussi diversi.

2. Scelta dei cammini: SSP determina il flusso minimo calcolando cammini augmenting successivi, mentre CC si concentra sulla cancellazione di cicli negativi. Le scelte dei cammini possono variare tra i due algoritmi, portando a flussi diversi.

3. Cicli negativi: CC si concentra sulla cancellazione dei cicli negativi all'interno del grafo residuo, mentre SSP calcola cammini augmenting successivi. La scoperta di cicli negativi e la loro cancellazione può variare tra le implementazioni.

4. Precisione numerica: Alcune implementazioni possono gestire la precisione numerica in modo diverso, il che potrebbe influenzare i risultati finali.

5. Condizioni di arresto: Le implementazioni possono avere criteri di arresto diversi per determinare quando il flusso minimo è stato trovato.

6. Strutture dati e ottimizzazioni: Le implementazioni possono utilizzare diverse strutture dati o ottimizzazioni, influenzando l'efficienza e i risultati.

Pertanto, se si utilizzano diverse implementazioni o si apportano modifiche specifiche all'algoritmo, è possibile ottenere risultati diversi per lo stesso grafo. È importante assicurarsi che le implementazioni siano corrette e confrontare i risultati in base alle specifiche esigenze del problema o dell'applicazione.